TODO: use [this](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb) to add multi-label classification

TODO: add extend data3/data.json with better data in the same format that actually makes sense.

## Imports and Setup

In [1]:
!pip install transformers -Uqq
!pip install sklearn -Uqq
!pip install datasets -Uqq
!pip install torch -Uqq
!pip install numpy -Uqq
!pip install evaluate -Uqq

In [2]:
import evaluate
import numpy as np
import torch
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Loading Dataset

In [3]:
dataset = load_dataset("json", data_files="data3/data.json", field="data")
dataset

Using custom data configuration default-6225a49313d03ca2
Found cached dataset json (/home/xt0r3/.cache/huggingface/datasets/json/default-6225a49313d03ca2/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'agency', 'humanComparison', 'hyperbole', 'historyComparison', 'unjustClaims', 'deepSounding', 'sceptics', 'deEmphasize', 'performanceNumber', 'inscrutable', 'objective'],
        num_rows: 100
    })
})

In [4]:
dataset["train"][0:3]

{'text': ['asdasd',
  'Agents unjustly claim history deep deemphasize objectivel',
  'Agents unjustly claim history deep deemphasize objectivel'],
 'agency': [True, True, True],
 'humanComparison': [False, False, False],
 'hyperbole': [False, False, False],
 'historyComparison': [True, True, True],
 'unjustClaims': [False, False, False],
 'deepSounding': [True, True, True],
 'sceptics': [False, False, False],
 'deEmphasize': [True, True, True],
 'performanceNumber': [False, False, False],
 'inscrutable': [False, False, False],
 'objective': [True, True, True]}

In [5]:
labels = [label for label in dataset["train"].features.keys() if label not in ["text"]]
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}
labels

['agency',
 'humanComparison',
 'hyperbole',
 'historyComparison',
 'unjustClaims',
 'deepSounding',
 'sceptics',
 'deEmphasize',
 'performanceNumber',
 'inscrutable',
 'objective']

## Preprocess Data, Create Train/Test Split

In [6]:
# dataset = dataset.class_encode_column("label")
dataset = dataset["train"].train_test_split(
    test_size=0.2
)  # , stratify_by_column="label")
#
# dataset

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def preprocess_data(examples):
    # take a batch of texts
    text = examples["text"]
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

In [8]:
encoded_dataset = dataset.map(
    preprocess_data, batched=True, remove_columns=dataset["train"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### Verify dataset

In [9]:
example = encoded_dataset["train"][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [10]:
tokenizer.decode(example["input_ids"])

'[CLS] Agents unjustly claim history deep deemphasize objectivel [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [11]:
example["labels"]

[1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0]

In [12]:
[id2label[idx] for idx, label in enumerate(example["labels"]) if label == 1.0]

['agency', 'historyComparison', 'deepSounding', 'deEmphasize', 'objective']

In [13]:
encoded_dataset.set_format("torch")

In [14]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 80
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20
    })
})

In [15]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [16]:
encoded_dataset['train']['input_ids'][0]

tensor([  101, 28052,  8362,  9380,  2050,  1193,  3548,  1607,  1996,  1260,
         5521, 20695,  5053,  3171,  7649,  1233,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

## Load Pre-Trained Model

In [17]:
# use_fast uses fast tokenizers backed by rust. Remove it if it causes errors
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(labels),
    problem_type="multi_label_classification",
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

### Verify data-model interaction

In [18]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7063, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1439,  0.4134,  0.0145,  0.5371,  0.0989, -0.3016,  0.3059, -0.0187,
          0.2124,  0.0336,  0.7427]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## Define Metrics

In [19]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average="micro")
    roc_auc = roc_auc_score(y_true, y_pred, average="micro")
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {"f1": f1_micro_average, "roc_auc": roc_auc, "accuracy": accuracy}
    return metrics


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds, labels=p.label_ids)
    return result

## Train the Model

In [20]:
batch_size = 1
metric_name = "f1"

In [21]:
training_args = TrainingArguments(
    f"bert-finetuned-multi-label-ai-hype",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True,  # TODO: enable once model seems good
)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics,
)

In [23]:
trainer.train()

/home/xt0r3/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 80
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 400
  Number of trainable parameters = 108318731


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.097095,1.000000,1.000000,1.000000
2,No log,0.031006,1.000000,1.000000,1.000000
3,No log,0.019973,1.000000,1.000000,1.000000
4,No log,0.016568,1.000000,1.000000,1.000000
5,No log,0.015587,1.000000,1.000000,1.000000


***** Running Evaluation *****
  Num examples = 20
  Batch size = 1
Saving model checkpoint to bert-finetuned-multi-label-ai-hype/checkpoint-80
Configuration saved in bert-finetuned-multi-label-ai-hype/checkpoint-80/config.json
Model weights saved in bert-finetuned-multi-label-ai-hype/checkpoint-80/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20
  Batch size = 1
Saving model checkpoint to bert-finetuned-multi-label-ai-hype/checkpoint-160
Configuration saved in bert-finetuned-multi-label-ai-hype/checkpoint-160/config.json
Model weights saved in bert-finetuned-multi-label-ai-hype/checkpoint-160/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20
  Batch size = 1
Saving model checkpoint to bert-finetuned-multi-label-ai-hype/checkpoint-240
Configuration saved in bert-finetuned-multi-label-ai-hype/checkpoint-240/config.json
Model weights saved in bert-finetuned-multi-label-ai-hype/checkpoint-240/pytorch_model.bin
***** Running Evaluation *****
  Num exa

TrainOutput(global_step=400, training_loss=0.0837978744506836, metrics={'train_runtime': 38.1198, 'train_samples_per_second': 10.493, 'train_steps_per_second': 10.493, 'total_flos': 26313231667200.0, 'train_loss': 0.0837978744506836, 'epoch': 5.0})

## Upload the Model

In [24]:
# trainer.push_to_hub()